## Imports

In [49]:
from scipy import stats
from statsmodels.distributions.empirical_distribution import ECDF

import mxnet as mx
import numpy as np
from mxnet import nd, autograd, gluon

# three customized modules
from labelshift import *
from utils4gluon import *
from data_shift import *
from data import *

np.random.seed(112358)

## Configurations

In [68]:
dataset_name = 'cifar10' # choices: 'mnist', 'cifar10'

################################################
# Decide whether to tweak the training dist and how
################################################
tweak_train = True # options include 

# UNCOMMENT to MANUALLY set training label distribution
p_P = [.1, .1, .1, .1 ,.1 ,.1, .1, .1, .1, .1]

# UNCOMMMENT to sample label distribution from Dirchlet w
# alpha = 1.
# p_P = np.random.dirichlet([1.]*10)


################################################
# Decide whether to tweak the test dist and how
################################################
tweak_test = True

# UNCOMMENT to MANUALLY set training label distribution
# p_Q = [.91, .01, .01, .01 ,.01 ,.01, .01, .01, .01, .01]

# UNCOMMENT to sample label distribution from Dirchlet w
alpha = .01
p_Q = np.random.dirichlet([1.]*10)

num_train_samples = 30000
num_val_samples = 30000
num_test_samples = 10000

################################################
# Neural network configurations
################################################
num_hidden = 256
epochs = 5

## Prepare the data

In [69]:
# set the context
ctx = mx.gpu()

# load the dataset
X, y, Xtest, ytest = load_data(dataset_name)
    
batch_size = 64
n = X.shape[0]
dfeat = np.prod(X.shape[1:])

# NOTE FOR IMPROVEMENT: eventually this should be returned by the data library
num_labels = 10

################################################
# Random permutation of the data
################################################

rand_idx = np.random.permutation(n)
X = X[rand_idx,...]
y = y[rand_idx]

################################################
#  First split examples between train and validation
################################################
num = 2  
Xtrain_source = X[:(n//num),:,:,:]
ytrain_source = y[:(n//num)]
Xval_source = X[(n//num):(2*n//num),:,:,:]
yval_source = y[(n//num):(2*n//num):]


################################################
#  Set the label distribution at train time
################################################
if tweak_train:
    print("Sampling training and validation data from p_P")
    print("Current p_P: ", p_P)

    Xtrain, ytrain = tweak_dist(Xtrain_source, ytrain_source, num_labels, num_train_samples, p_P)
    Xval, yval = tweak_dist(Xval_source, yval_source, num_labels, num_val_samples, p_P)
else:
    Xtrain, ytrain = Xtrain_source, ytrain_source
    Xval, yval = Xval_source, yval_source

################################################
#  Set the label distribution for test data
################################################
if tweak_test:
    print("Sampling test data from p_Q")
    print("Current p_Q: ", p_Q)
    Xtest, ytest = tweak_dist(Xtest, ytest, num_labels, num_test_samples, p_Q)

Sampling training and validation data from p_P
Current p_P:  [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
Sampling test data from p_Q
Current p_Q:  [ 0.05192518  0.403422    0.0252058   0.00061585  0.15051928  0.05287386
  0.23776149  0.0213737   0.00853258  0.04777026]


In [70]:
print(Xtrain.shape)
print(Xval.shape)
print(Xtest.shape)

(30000, 3, 32, 32)
(30000, 3, 32, 32)
(10000, 3, 32, 32)


In [71]:
# debugging code
print(nd.one_hot(nd.array(ytrain), 10).sum(axis=0) / len(ytrain))
print(nd.one_hot(nd.array(yval), 10).sum(axis=0) / len(yval))
print(nd.one_hot(nd.array(ytest), 10).sum(axis=0) / len(ytest))


[ 0.1023      0.1011      0.09733333  0.09956667  0.10146666  0.10123333
  0.10076667  0.1002      0.10016666  0.09586667]
<NDArray 10 @cpu(0)>

[ 0.09883333  0.09903333  0.09916667  0.10156666  0.1012      0.09963334
  0.10003334  0.0995      0.09936666  0.10166667]
<NDArray 10 @cpu(0)>

[ 0.0534  0.4064  0.0269  0.0005  0.148   0.0542  0.2343  0.0227  0.0072
  0.0464]
<NDArray 10 @cpu(0)>


##  Train a Classifier

In [74]:
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net.add(gluon.nn.Dense(num_labels))

net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1})

# Training


# Prediction
ypred_s, ypred_s_soft = predict_all(Xval, net, ctx, dfeat)
ypred_t, ypred_t_soft = predict_all(Xtest, net, ctx, dfeat)


# Converting to numpy array for later convenience
ypred_s= ypred_s.asnumpy()
ypred_s_soft = ypred_s_soft.asnumpy()
ypred_t=ypred_t.asnumpy()

Epoch 0. Loss: 1.87659304347, Train_acc 0.339885394456, Validation_acc 0.329257729211
Epoch 1. Loss: 1.72369736903, Train_acc 0.389692164179, Validation_acc 0.371735074627
Epoch 2. Loss: 1.6248598581, Train_acc 0.419542910448, Validation_acc 0.392390724947
Epoch 3. Loss: 1.54958461123, Train_acc 0.438699360341, Validation_acc 0.404750799574
Epoch 4. Loss: 1.48154066063, Train_acc 0.469049840085, Validation_acc 0.412646588486


## Estimate Wt and Py

In [75]:
wt = estimate_labelshift_ratio(yval, ypred_s, ypred_t,num_labels)

Py_est = estimate_target_dist(wt, yval,num_labels)

Py_true =calculate_marginal(ytest,num_labels)
Py_base =calculate_marginal(yval,num_labels)

wt_true = Py_true/Py_base

print(np.concatenate((wt,wt_true),axis=1))
print(np.concatenate((Py_est,Py_true),axis=1))

print("||wt - wt_true||^2  = " + repr(np.sum((wt-wt_true)**2)/np.linalg.norm(wt_true)**2))
print("KL(Py_est|| Py_true) = " + repr(stats.entropy(Py_est,Py_base)))

[[ 0.61568333  0.54030354]
 [ 4.11867255  4.1036688 ]
 [ 0.78713783  0.2712605 ]
 [ 0.17984028  0.00492287]
 [ 1.32314199  1.46245059]
 [ 0.0796509   0.54399465]
 [ 2.28978031  2.34221926]
 [ 0.11646368  0.2281407 ]
 [ 0.17112826  0.07245891]
 [ 0.34874449  0.45639344]]
[[ 0.06085004  0.0534    ]
 [ 0.40788587  0.4064    ]
 [ 0.07805784  0.0269    ]
 [ 0.01826578  0.0005    ]
 [ 0.13390197  0.148     ]
 [ 0.00793588  0.0542    ]
 [ 0.22905436  0.2343    ]
 [ 0.01158814  0.0227    ]
 [ 0.01700444  0.0072    ]
 [ 0.03545569  0.0464    ]]
||wt - wt_true||^2  = 0.022613514546137293
KL(Py_est|| Py_true) = array([ 0.61273163])


## Solve weighted ERM and compare to previously trained models

In [76]:
data_test = mx.io.NDArrayIter(Xtest, ytest, batch_size, shuffle=False)

acc_unweighted =  evaluate_accuracy(data_test, net, ctx, dfeat) # in fact, drawing confusion matrix maybe more informative

print("Accuracy unweighted", acc_unweighted)

training_weights=np.maximum(wt, 0)
wt_ndarray = nd.array(training_weights,ctx=ctx)


weightfunc = lambda x,y: wt_ndarray[y.asnumpy().astype(int)]

# Train a model using the following!
net2 = gluon.nn.Sequential()
with net2.name_scope():
    net2.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net2.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net2.add(gluon.nn.Dense(num_labels))

net2.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)
trainer2 = gluon.Trainer(net2.collect_params(), 'sgd', {'learning_rate': .1})
epochs2 = 5

# Training
weighted_train(net2, softmax_cross_entropy, trainer2, Xtrain, ytrain, Xval, yval, ctx, dfeat, epoch=epochs2, weightfunc=weightfunc)

data_test.reset()
acc_weighted = evaluate_accuracy(data_test, net2, ctx, dfeat)

print("Accuracy weighted", acc_weighted)

Accuracy unweighted 0.403761942675
Epoch 0. Loss: 1.41994833343, Train_acc 0.219549573561, Validation_acc 0.210920842217
Epoch 1. Loss: 1.27097178876, Train_acc 0.250732942431, Validation_acc 0.238506130064
Epoch 2. Loss: 1.18565286155, Train_acc 0.267157515991, Validation_acc 0.25006663113
Epoch 3. Loss: 1.11673102186, Train_acc 0.261293976546, Validation_acc 0.246501865672
Epoch 4. Loss: 1.06252223125, Train_acc 0.28158315565, Validation_acc 0.26012793177
Accuracy weighted 0.587878184713
